In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your dataset
data = pd.read_csv('xa.s12.00.mhz.1970-01-19HR00_evid00002example.csv')

# Display the first few rows to understand its structure
print(data.head())

# The dataset tested had the columns named as "time_rel(sec)" and veclocity(sec), change as required  
time_column = 'time_rel(sec)'  
value_column = 'velocity(m/s)'  

# Plot the original data
plt.figure(figsize=(12, 6))
plt.plot(data[time_column], data[value_column], color='blue', label='Seismic Data')
plt.title('Seismic Data Plot')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import pywt

# Apply wavelet transform to the relevant column (adjust the column name as needed)
coeffs = pywt.wavedec(data['velocity(m/s)'], 'db4', level=6)

# Create time absolute and relative columns
time_abs = data['time_abs(%Y-%m-%dT%H:%M:%S.%f)']
time_rel = (pd.to_datetime(time_abs) - pd.to_datetime(time_abs.iloc[0])).dt.total_seconds()

# Create a dictionary to store wavelet coefficients by level (VERYYYYYY IMPORTANT)
wavelet_data = {
    'time_abs(%Y-%m-%dT%H:%M:%S.%f)': time_abs,
    'time_rel(sec)': time_rel
}

# Add each level's wavelet coefficients to the DataFrame
for i, coeff in enumerate(coeffs):
    wavelet_data[f'wavelet_coefficients_level_{i}'] = np.pad(coeff, (0, len(time_abs) - len(coeff)), mode='constant', constant_values=np.nan)

# Create the DataFrame with the wavelet coefficients organized by level
wavelet_df = pd.DataFrame(wavelet_data)

# Save the DataFrame to a CSV file
wavelet_df.to_csv('wavelet_coefficients_by_level.csv', index=False)

print("Wavelet coefficients by level saved to 'wavelet_coefficients_by_level.csv'.")


In [ ]:
import matplotlib.pyplot as plt

# Create a figure to plot the coefficients
plt.figure(figsize=(14, 10))

# Plot each level of coefficients
for i, coeff in enumerate(coeffs):
    plt.subplot(len(coeffs), 1, i + 1)
    plt.plot(coeff, label=f'Wavelet Coefficients - Level {i}')
    plt.title(f'Wavelet Coefficients - Level {i}')
    plt.xlabel('Sample Index')
    plt.ylabel('Coefficient Value')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Load the wavelet coefficients CSV file
wavelet_df = pd.read_csv('wavelet_coefficients_by_level.csv')

# List to hold coefficient arrays for each level
coeffs = []
max_levels = 7  # Assuming you want to check for levels 0 to 6 (the 'resolution' so to say of this program goes from 0-6)

# Check and create arrays for each level
for level in range(max_levels):
    col_name = f'wavelet_coefficients_level_{level}'
    
    if col_name in wavelet_df.columns:
        # Get the coefficient array for this level
        coeff_array = wavelet_df[col_name].values
        
        # Find the last non-empty value's index
        last_valid_index = np.where(~np.isnan(coeff_array))[0][-1] if np.any(~np.isnan(coeff_array)) else -1
        
        if last_valid_index >= 0:
            # Slice the coefficient array to keep only the valid values
            trimmed_coeff_array = coeff_array[:last_valid_index + 1]
            coeffs.append(trimmed_coeff_array)
            print(f'Shape of {col_name}: {trimmed_coeff_array.shape}')  # Print the shape of each coefficient array
        else:
            print(f'No valid values found in {col_name}.')
    else:
        print(f'Column {col_name} does not exist in the DataFrame.')

# At this point, you have coeffs as a list containing arrays for each level
#Arrays are padded to avoid dimension errors

In [ ]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

# Load wavelet coefficients
wavelet_df = pd.read_csv('wavelet_coefficients_by_level.csv')

# Extract coefficients for levels 4, 5, and 6 (most common for quake levels)
level_4 = wavelet_df['wavelet_coefficients_level_4'].dropna().values
level_5 = wavelet_df['wavelet_coefficients_level_5'].dropna().values
level_6 = wavelet_df['wavelet_coefficients_level_6'].dropna().values

# Ensure the same length for all levels (padding)
min_length = min(len(level_4), len(level_5), len(level_6))
level_4 = level_4[:min_length]
level_5 = level_5[:min_length]
level_6 = level_6[:min_length]

# Combine coefficients
combined_coeffs = level_4 + level_5 + level_6

# Smooth the combined data
smoothed_data = gaussian_filter1d(combined_coeffs, sigma=2)

# Create time series for plotting
time_rel = wavelet_df['time_rel(sec)'].dropna().values[:min_length]

# Set a higher threshold for peak detection
mean_value = np.mean(smoothed_data)
std_value = np.std(smoothed_data)

######### for the start point
# Increase the multiplier significantly to further reduce false positives
threshold_value = mean_value + 4 * std_value  # Try increasing to 4 or 5 (Change as required)
minimum_peak_height = threshold_value  # Set a minimum height for detected peaks
minimum_peak_width = 10  # Set minimum width for detected peaks

# Find peaks to mark the start of seismic events
peaks, properties = find_peaks(smoothed_data, height=minimum_peak_height, distance=50, width=minimum_peak_width)

# Check if any peaks are found and use the first one for marking
event_start = peaks[0] if len(peaks) > 0 else None

########## for the end point
# Set a threshold for end detection
threshold = mean_value - 2.5 * std_value  # Adjust this value to your data characteristics (same as above)

# Start from the end of the signal and look for the end of the event
for i in range(min_length - 1, 0, -1):
    # Check if the amplitude is below the threshold for stability
    if smoothed_data[i] < threshold:
        event_end = i
        break
else:
    event_end = None

# Plot the smoothed data and mark the event start and end
plt.figure(figsize=(14, 7))
plt.plot(time_rel, smoothed_data, label='Smoothed Wavelet Coefficients (Levels 4, 5, 6)', color='blue')
if event_start is not None:
    plt.axvline(x=time_rel[event_start], color='red', label='Suspected Event Start', linestyle='--')
if event_end is not None:
    plt.axvline(x=time_rel[event_end], color='green', label='Suspected Event End', linestyle='--')
plt.title('Seismic Event Analysis')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid()
plt.show()

# Save the smoothed signal and time to CSV
output_df = pd.DataFrame({
    'time_rel(sec)': time_rel,
    'smoothed_wavelet_coefficients': smoothed_data
})

# Save to CSV
output_df.to_csv('smoothed_wavelet_coefficients.csv_with_start_and_end', index=False)

print("Smoothed wavelet coefficients saved to 'smoothed_wavelet_coefficients.csv_with_start_and_end'.")

# Check the x-values where the event start and end lines are drawn
if event_start is not None:
    event_start_time = time_rel[event_start]
    print(f"The suspected start of the seismic event is at: {event_start_time} seconds")
else:
    print("No event start detected.")

if event_end is not None:
    event_end_time = time_rel[event_end]
    print(f"The suspected end of the seismic event is at: {event_end_time} seconds")
else:
    print("No event end detected.")

In [ ]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

# Load wavelet coefficients
wavelet_df = pd.read_csv('wavelet_coefficients_by_level.csv')

# Extract coefficients for levels 4, 5, and 6
level_4 = wavelet_df['wavelet_coefficients_level_4'].dropna().values
level_5 = wavelet_df['wavelet_coefficients_level_5'].dropna().values
level_6 = wavelet_df['wavelet_coefficients_level_6'].dropna().values

# Ensure the same length for all levels (padding)
min_length = min(len(level_4), len(level_5), len(level_6))
level_4 = level_4[:min_length]
level_5 = level_5[:min_length]
level_6 = level_6[:min_length]

# Combine coefficients
combined_coeffs = level_4 + level_5 + level_6

# Smooth the combined data
smoothed_data = gaussian_filter1d(combined_coeffs, sigma=2)

# Create time series for plotting
time_rel = wavelet_df['time_rel(sec)'].dropna().values[:min_length]

# Set a higher threshold for peak detection
mean_value = np.mean(smoothed_data)
std_value = np.std(smoothed_data)

# Increase the multiplier significantly to further reduce false positives
threshold_value = mean_value + 4 * std_value  # Try increasing to 4 or 5
minimum_peak_height = threshold_value  # Set a minimum height for detected peaks
minimum_peak_width = 10  # Set minimum width for detected peaks

# Find peaks to mark the start of seismic events
peaks, properties = find_peaks(smoothed_data, height=minimum_peak_height, distance=50, width=minimum_peak_width)

# Check if any peaks are found and use the first one for marking
event_start = peaks[0] if len(peaks) > 0 else None

# Set a threshold for end detection
threshold = mean_value - 2.5 * std_value  # Adjust this value to your data characteristics

# Start from the end of the signal and look for the end of the event
for i in range(min_length - 1, 0, -1):
    # Check if the amplitude is below the threshold for stability
    if smoothed_data[i] < threshold:
        event_end = i
        break
else:
    event_end = None

# Create a modified data array to replace data outside the start and end
modified_data = np.zeros_like(smoothed_data)

if event_start is not None and event_end is not None:
    # Keep the values within the event start and end range
    modified_data[event_start:event_end + 1] = smoothed_data[event_start:event_end + 1]

# Create a time series for plotting the modified data
modified_time = time_rel  # Keep original time for plotting

# Remove rows where wavelet coefficients are zero
filtered_df = pd.DataFrame({
    'time_rel(sec)': modified_time,
    'modified_wavelet_coefficients': modified_data
})

# Filter out rows where modified_wavelet_coefficients are zero
filtered_df = filtered_df[filtered_df['modified_wavelet_coefficients'] != 0]

# Plot the modified data with a horizontal line at y=0
plt.figure(figsize=(14, 7))
plt.plot(modified_time, modified_data, label='Seismic Event Data', color='blue')  # Changed to blue
plt.axhline(y=0, color='black', linestyle='--')  # Removed label for y=0 line
if event_start is not None:
    plt.axvline(x=modified_time[event_start], color='red', label='Suspected Event Start', linestyle='--')
if event_end is not None:
    plt.axvline(x=modified_time[event_end], color='green', label='Suspected Event End', linestyle='--')

plt.title('Seismic Event Analysis - Modified Data')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid()
plt.show()

# Save the filtered signal and time to CSV
filtered_df.to_csv('modified_wavelet_coefficients.csv', index=False)
print("Modified wavelet coefficients saved to 'modified_wavelet_coefficients.csv'.")

# Check the x-values where the event start and end lines are drawn
if event_start is not None:
    event_start_time = time_rel[event_start]
    print(f"The suspected start of the seismic event is at: {event_start_time} seconds")
else:
    print("No event start detected.")

if event_end is not None:
    event_end_time = time_rel[event_end]
    print(f"The suspected end of the seismic event is at: {event_end_time} seconds")
else:
    print("No event end detected.")

In [ ]:
###########################Fourier is still amazing#############################


In [ ]:
###########################SONIFICATION PART, OPTIONAL -- TO BE USED FOR AUDITORY DATA ANALYSIS#############################


In [ ]:
import pandas as pd #Importing pandas for data-frame execution
filename = 'xa.s12.00.mhz.1970-01-19HR00_evid00002example' #Add your file name
df = pd.read_csv(filename + '.csv')
n_slices = len(df) #No, of slices of the image (automates regradless the number of slices)
print (n_slices, 'time_rel(sec)')
df.head() #Ensuring data is in place
#Sequence can be thought as a progression of time

In [ ]:
#plotting the data gives a visual feedback about the end result, not necessary but recommended
#Substitute red, blue, green, stars for your own fields, the code is just so flexible!
import matplotlib.pylab as plt 
sequence = df['time_rel(sec)'].values
red = df['velocity(m/s)'].values
#green = df['green'].values
#blue = df ['blue'].values

plt.subplot(2, 2, 1)
plt.scatter(sequence, red)
plt.xlabel('time_rel(sec)')
plt.ylabel('velocity(m/s)')
plt.show()

#plt.subplot(2, 2, 2)
#plt.scatter(sequence, green)
#plt.xlabel('sequence')
#plt.ylabel('green')
#plt.show()

#plt.subplot(2, 2, 3)
#plt.scatter(sequence, blue)
#plt.xlabel('sequence')
#plt.ylabel('blue')
#plt.show()


In [ ]:
#A mapping function, performs linear mapping or interpolation. Use to map a variable to the sound parameter
def map_value(value, min_value, max_value, min_result, max_result):
 result = min_result + (value - min_value)/(max_value - min_value)*(max_result - min_result)
 return result

In [ ]:
sequence_per_beat = 1 #Highly depends on the desired output, this example goes for 1 to depict 1 beat eper second
s_data = sequence/sequence_per_beat
duration_beats = max(s_data)
duration_beats

In [ ]:
bpm = 60  
duration_sec = duration_beats*60/bpm #Scales the duration to seconds for ease
print('Duration:', duration_sec, 'seconds')

#These graphs represent the time against varible.
plt.subplot(2, 2, 1)
plt.scatter(s_data, red)
plt.xlabel('time [beats]')
plt.ylabel('red')
plt.show()

#plt.subplot(2, 2, 1)
#plt.scatter(s_data, green)
#plt.xlabel('time [beats]')
#plt.ylabel('green')
#plt.show()

#plt.subplot(2, 2, 1)
#plt.scatter(s_data, blue)
#plt.xlabel('time [beats]')
#plt.ylabel('blue')
#plt.show()

In [ ]:
#Scaling values to control the data in the MIDI production
ry_data = map_value(red, min(red), max(red), 0, 1)

ry_scale = 0.5 

ry_data = ry_data**ry_scale

plt.subplot(2 ,2, 2)
plt.scatter(sequence, ry_data, s=50*ry_data)
plt.xlabel('sequence')
plt.ylabel('y data [normalized]')
plt.show()

#gy_data = map_value(green, min(green), max(green), 0, 1)

#gy_scale = 0.5 

#gy_data = gy_data**gy_scale

#plt.subplot(2 ,2, 3)
#plt.scatter(sequence, gy_data, s=50*gy_data)
#plt.xlabel('sequence')
#plt.ylabel('y data [normalized]')
#plt.show()

#by_data = map_value(blue, min(blue), max(blue), 0, 1)

#by_scale = 0.5 

#by_data = by_data**by_scale

#plt.subplot(2 ,2, 4)
#plt.scatter(sequence, by_data, s=50*by_data)
#plt.xlabel('sequence')
#plt.ylabel('y data [normalized]')
#plt.show()

In [ ]:
from midiutil.MidiFile import MIDIFile

# Define the scale using note names and their corresponding MIDI numbers
note_names = ['C2', 'D2', 'E2', 'G2', 'A2',
              'C3', 'D3', 'E3', 'G3', 'A3',
              'C4', 'D4', 'E4', 'G4', 'A4',
              'C5', 'D5', 'E5', 'G5', 'A5']

# Use a dictionary for MIDI note conversion
note_to_midi = {
    'C2': 36, 'D2': 38, 'E2': 40, 'G2': 43, 'A2': 45,
    'C3': 48, 'D3': 50, 'E3': 52, 'G3': 55, 'A3': 57,
    'C4': 60, 'D4': 62, 'E4': 64, 'G4': 67, 'A4': 69,
    'C5': 72, 'D5': 74, 'E5': 76, 'G5': 79, 'A5': 81
}

# Convert notes to MIDI numbers
note_midis = [note_to_midi[n] for n in note_names]

n_notes = len(note_midis)
print('Resolution:', n_notes, 'notes')


In [ ]:
pitch1_data = [] #red is pitch1 data
for i in range(n_slices):
    note_pitch1 = round(map_value(ry_data[i], 0, 1, 0, n_notes-1))  
    pitch1_data.append(note_midis[note_pitch1])
plt.scatter(s_data, pitch1_data, s=50*ry_data)
plt.xlabel('time [beats]')
plt.ylabel('midi note numbers')
plt.show()

In [ ]:
import numpy as np
from midiutil import MIDIFile 

# Sample pitch1_data (replace this with your actual data)
pitch1_data = [60, 62, 64, 65, 67, 69, 71]  # Example MIDI note numbers
# Assume pitch2_data is empty or filled with zeros
pitch2_data = []  # This will be filled with zeros if it's empty

# Check if pitch2_data is empty and fill with zeros if it is
if len(pitch2_data) == 0:
    pitch2_data = [0] * len(pitch1_data)  # Fill with zeros

# Convert to NumPy arrays
pitch1_array = np.array(pitch1_data)
pitch2_array = np.array(pitch2_data)

# Calculate the average pitch
pitch_array = np.round((pitch1_array + pitch2_array) / 2)

# Set up MIDI file
bpm = 120  # Set your preferred BPM
filename = 'Original Data'  # Set your filename
n_slices = len(pitch1_array)  # Assuming both arrays have the same length

my_midi_file = MIDIFile(1)  
my_midi_file.addTempo(track=0, time=0, tempo=bpm) 

# Add notes to the MIDI file using pitch1_array (you can also use pitch_array if needed)
for i in range(n_slices):
    my_midi_file.addNote(track=0, channel=0, pitch=int(pitch_array[i]), time=i, duration=2, volume=100)

# Save MIDI file
with open(filename + '.mid', "wb") as f:
    my_midi_file.writeFile(f)

print(f"MIDI file saved as '{filename}.mid'")


In [ ]:
import pandas as pd #Importing pandas for data-frame execution
filename = 'modified_wavelet_coefficients' #Add your file name
df = pd.read_csv(filename + '.csv')
n_slices = len(df) #No, of slices of the image (automates regradless the number of slices)
print (n_slices, 'time_rel(sec)')
df.head() #Ensuring data is in place
#Sequence can be thought as a progression of time

In [ ]:
#plotting the data gives a visual feedback about the end result, not necessary but recommended
#Substitute red, blue, green, stars for your own fields, the code is just so flexible!
import matplotlib.pylab as plt 
sequence = df['time_rel(sec)'].values
red = df['modified_wavelet_coefficients'].values
#green = df['green'].values
#blue = df ['blue'].values

plt.subplot(2, 2, 1)
plt.scatter(sequence, red)
plt.xlabel('time_rel(sec)')
plt.ylabel('modified_wavelet_coefficients')
plt.show()

#plt.subplot(2, 2, 2)
#plt.scatter(sequence, green)
#plt.xlabel('sequence')
#plt.ylabel('green')
#plt.show()

#plt.subplot(2, 2, 3)
#plt.scatter(sequence, blue)
#plt.xlabel('sequence')
#plt.ylabel('blue')
#plt.show()


In [ ]:
#A mapping function, performs linear mapping or interpolation. Use to map a variable to the sound parameter
def map_value(value, min_value, max_value, min_result, max_result):
 result = min_result + (value - min_value)/(max_value - min_value)*(max_result - min_result)
 return result

In [ ]:
sequence_per_beat = 1 #Highly depends on the desired output, this example goes for 1 to depict 1 beat eper second
s_data = sequence/sequence_per_beat
duration_beats = max(s_data)
duration_beats

In [ ]:
bpm = 60  
duration_sec = duration_beats*60/bpm #Scales the duration to seconds for ease
print('Duration:', duration_sec, 'seconds')

#These graphs represent the time against varible.
plt.subplot(2, 2, 1)
plt.scatter(s_data, red)
plt.xlabel('time [beats]')
plt.ylabel('red')
plt.show()

#plt.subplot(2, 2, 1)
#plt.scatter(s_data, green)
#plt.xlabel('time [beats]')
#plt.ylabel('green')
#plt.show()

#plt.subplot(2, 2, 1)
#plt.scatter(s_data, blue)
#plt.xlabel('time [beats]')
#plt.ylabel('blue')
#plt.show()

In [ ]:
#Scaling values to control the data in the MIDI production
ry_data = map_value(red, min(red), max(red), 0, 1)

ry_scale = 0.5 

ry_data = ry_data**ry_scale

plt.subplot(2 ,2, 2)
plt.scatter(sequence, ry_data, s=50*ry_data)
plt.xlabel('sequence')
plt.ylabel('y data [normalized]')
plt.show()

#gy_data = map_value(green, min(green), max(green), 0, 1)

#gy_scale = 0.5 

#gy_data = gy_data**gy_scale

#plt.subplot(2 ,2, 3)
#plt.scatter(sequence, gy_data, s=50*gy_data)
#plt.xlabel('sequence')
#plt.ylabel('y data [normalized]')
#plt.show()

#by_data = map_value(blue, min(blue), max(blue), 0, 1)

#by_scale = 0.5 

#by_data = by_data**by_scale

#plt.subplot(2 ,2, 4)
#plt.scatter(sequence, by_data, s=50*by_data)
#plt.xlabel('sequence')
#plt.ylabel('y data [normalized]')
#plt.show()

In [ ]:
from midiutil.MidiFile import MIDIFile

# Define the scale using note names and their corresponding MIDI numbers
note_names = ['C2', 'D2', 'E2', 'G2', 'A2',
              'C3', 'D3', 'E3', 'G3', 'A3',
              'C4', 'D4', 'E4', 'G4', 'A4',
              'C5', 'D5', 'E5', 'G5', 'A5']

# Use a dictionary for MIDI note conversion
note_to_midi = {
    'C2': 36, 'D2': 38, 'E2': 40, 'G2': 43, 'A2': 45,
    'C3': 48, 'D3': 50, 'E3': 52, 'G3': 55, 'A3': 57,
    'C4': 60, 'D4': 62, 'E4': 64, 'G4': 67, 'A4': 69,
    'C5': 72, 'D5': 74, 'E5': 76, 'G5': 79, 'A5': 81
}

# Convert notes to MIDI numbers
note_midis = [note_to_midi[n] for n in note_names]

n_notes = len(note_midis)
print('Resolution:', n_notes, 'notes')


In [ ]:
pitch1_data = [] #red is pitch1 data
for i in range(n_slices):
    note_pitch1 = round(map_value(ry_data[i], 0, 1, 0, n_notes-1))  
    pitch1_data.append(note_midis[note_pitch1])
plt.scatter(s_data, pitch1_data, s=50*ry_data)
plt.xlabel('time [beats]')
plt.ylabel('midi note numbers')
plt.show()

In [ ]:
import numpy as np
from midiutil import MIDIFile 

# Sample pitch1_data (replace this with your actual data)
pitch1_data = [60, 62, 64, 65, 67, 69, 71]  # Example MIDI note numbers
# Assume pitch2_data is empty or filled with zeros
pitch2_data = []  # This will be filled with zeros if it's empty

# Check if pitch2_data is empty and fill with zeros if it is
if len(pitch2_data) == 0:
    pitch2_data = [0] * len(pitch1_data)  # Fill with zeros

# Convert to NumPy arrays
pitch1_array = np.array(pitch1_data)
pitch2_array = np.array(pitch2_data)

# Calculate the average pitch
pitch_array = np.round((pitch1_array + pitch2_array) / 2)

# Set up MIDI file
bpm = 120  # Set your preferred BPM
filename = 'Processed data'  # Set your filename
n_slices = len(pitch1_array)  # Assuming both arrays have the same length

my_midi_file = MIDIFile(1)  
my_midi_file.addTempo(track=0, time=0, tempo=bpm) 

# Add notes to the MIDI file using pitch1_array (you can also use pitch_array if needed)
for i in range(n_slices):
    my_midi_file.addNote(track=0, channel=0, pitch=int(pitch_array[i]), time=i, duration=2, volume=100)

# Save MIDI file
with open(filename + '.mid', "wb") as f:
    my_midi_file.writeFile(f)

print(f"MIDI file saved as '{filename}.mid'")
